In [ ]:
# Cell 1 - Install required packages
!pip install numpy # For numerical operations and array manipulation
!pip install pillow # For handling images in Python
!pip install scikit-learn # For pre-processing utilities, like min-max normalisation

In [ ]:
# Cell 2 - Import packages 
import os # For interacting with the operating system
import cv2 # For handling grayscale and multi-channel TIFFS 
import random # To generate random numbers
import glob # For searching for specific file patterns e.g., .TIFF files
import numpy as np # For numerical operations and array manipulation
from PIL import Image # For handling images in Python
from sklearn.preprocessing import minmax_scale # Applies min-max normalisation (0-255 range) to images 

In [ ]:
#Cell 3 - Renaming files 
# === USER INPUT: folder containing your .tif files ===
FOLDER = "/User/path" #INPUT DIRECTORY # Edit to add the pathname to your treatment files

# Optional: set number of digits for the random code
NUM_DIGITS = 6  # e.g., 6 → filenames like 483920.tif

# Create a set to track used codes (to avoid duplicates)
used_codes = set()

def generate_code():
    """Generate a unique random number string of NUM_DIGITS length."""
    while True:
        code = str(random.randint(10**(NUM_DIGITS-1), 10**NUM_DIGITS - 1))
        if code not in used_codes:
            used_codes.add(code)
            return code

# Process all .tif files in the folder
for filename in os.listdir(FOLDER):
    if filename.lower().endswith(".tif"):
        old_path = os.path.join(FOLDER, filename)
        new_name = generate_code() + ".tif"
        new_path = os.path.join(FOLDER, new_name)

        os.rename(old_path, new_path)
        print(f" {filename} → {new_name}")

print("Renaming complete!")

In [ ]:
#Cell 4 - Min-Max Normalisation of images
# === USER INPUT: set your treatment folders here ===
FOLDER_1 = "/User/path"  # Add pathname of your Treatment_1 folder
FOLDER_2 = "/User/path"  # Add pathname of your Treatment_2 folder

# === OUTPUT BASE FOLDER ===
OUTPUT_DIR = "/User/path"  # Add pathname of your Output folder
os.makedirs(OUTPUT_DIR, exist_ok=True)

def normalize_with_sklearn(image_array):
    """ Normalise image using sklearn's minmax_scale to [0,255]."""
    flat = image_array.flatten().astype(float)
    scaled = minmax_scale(flat, feature_range=(0, 255))
    return scaled.reshape(image_array.shape).astype(np.uint8)

def process_folder(input_dir, output_base):
    output_folder = os.path.join(output_base, os.path.basename(input_dir) + "_norm")
    os.makedirs(output_folder, exist_ok=True)
    for file in os.listdir(input_dir):
        if file.lower().endswith(".tif"):
            img = cv2.imread(os.path.join(input_dir, file), cv2.IMREAD_UNCHANGED)
            norm_img = normalize_with_sklearn(img)
            cv2.imwrite(os.path.join(output_folder, file), norm_img)
    print(f"Normalized images saved to: {output_folder}")

# --- RUN ---
for folder in [FOLDER_1, FOLDER_2]:
    process_folder(folder, OUTPUT_DIR)